In [6]:
#Basic Tools
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import csv

In [7]:
#NLP Tools
from gensim import corpora, models, similarities, matutils
from sklearn import datasets
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from accessory_functions import preprocess_series_text, nltk_path
# logging for gensim (set to INFO)
# gensim
import gensim
from gensim import corpora, models, similarities, matutils
import logging
import csv
import nltk

In [25]:
#Machine Learning Tools
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import cross_val_score
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import log_loss
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import svm
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import f1_score
from sklearn import svm
from sklearn.naive_bayes import MultinomialNB
from sklearn.cross_validation import train_test_split
from sklearn import metrics

In [14]:
#abstract_df.pickle is a dataframe containing abstracts from 8 journals.
merged_df = pd.read_pickle('abstract_df.pickle')
merged_df.head()

,Journal,Abstract
0,1,This paper presents a new hybrid algorithm tha...
1,1,Electronic Commerce (EC) has offered a new cha...
2,1,Activity recognition in monitored environments...
3,1,"As is widely recognized, sign language recogni..."
4,1,This paper emphasizes on supporting internatio...


In [78]:
#Pre-processing
tfidf = TfidfVectorizer(stop_words="english", 
                        token_pattern="\\b[a-zA-Z][a-zA-Z]+\\b", 
                        min_df=10)
tfidf_vecs = tfidf.fit_transform(merged_df["Abstract"])

In [90]:
df1 = pd.DataFrame(tfidf_vecs.todense(), 
             columns=tfidf.get_feature_names()
            )

In [50]:
#Applying LSI(Latent Semantic Indexing) for Dimensionality Reduction
tfidf_corpus = matutils.Sparse2Corpus(tfidf_vecs.transpose())
id2word = dict((v, k) for k, v in tfidf.vocabulary_.items())
lsi = models.LsiModel(tfidf_corpus, id2word=id2word, num_topics=300)
lsi_corpus = lsi[tfidf_corpus]

In [55]:
ng_lsi = matutils.corpus2dense(lsi_corpus, num_terms=300).transpose()

In [61]:
#Train and test Data
X_train, X_test, y_train, y_test = train_test_split(ng_lsi, 
                                                    merged_df["Journal"], 
                                                    test_size=0.2)

In [110]:
#Logistic regression
lr_no_regularization = LogisticRegression(C = 100000000000000)
lr_no_regularization.fit(X_train,y_train)
y_pred= lr_no_regularization.predict(X_test)
logic_accuracy = metrics.accuracy_score(y_test,y_pred)

In [111]:
word_df = pd.DataFrame(ng_lsi)

In [80]:
#Adding features from LDA 
#Pre-Processing
count_vectorizer = CountVectorizer(ngram_range=(1, 3),  
                                   stop_words='english', token_pattern="\\b[a-z][a-z]+\\b")
count_vectorizer.fit(merged_df["Abstract"])
counts = count_vectorizer.transform(merged_df["Abstract"]).transpose()
pd.DataFrame(counts.transpose().todense(), 
             columns=count_vectorizer.get_feature_names()
            ).head()

,aa,aa diagnosability,aa diagnosability basis,aa diagnosability sdes,aa zz,aa zz lower,aaai,aaai providence,aaai providence ri,aac,...,zou fuzzy math,zpn,zpn zqm,zpn zqm distinct,zqm,zqm distinct,zqm distinct primes,zz,zz lower,zz lower case
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [116]:
corpus = matutils.Sparse2Corpus(counts)
id2word = dict((v, k) for k, v in count_vectorizer.vocabulary_.items())

In [82]:
#Applying LDA
lda = models.LdaModel(corpus=corpus, num_topics=5, id2word=id2word, passes=10)

In [83]:
lda_corpus = lda[corpus]
lda_docs = [doc for doc in lda_corpus]

In [112]:
topic_1 = []
topic_2 = []
topic_3 = []
topic_4 = []
topic_5 = []
for item in lda_docs:
    try:
        topic_1.append(item[0][1]) 
    except:
        topic_1.append(0)
    try:
        topic_2.append(item[1][1]) 
    except:
        topic_2.append(0)
    try:
        topic_3.append(item[2][1]) 
    except:
        topic_3.append(0)
    try:
        topic_4.append(item[3][1]) 
    except:
        topic_4.append(0)
    try:
        topic_5.append(item[4][1]) 
    except:
        topic_5.append(0)
word_df["Topic1"] = topic_1
word_df["Topic2"] = topic_2
word_df["Topic3"] = topic_3
word_df["Topic4"] = topic_4
word_df["Topic5"] = topic_5

In [115]:
word_df.head()

,0,1,2,3,4,5,6,7,8,9,...,295,296,297,298,299,Topic1,Topic2,Topic3,Topic4,Topic5
0,0.219627,0.099792,-0.050028,0.259227,-0.243618,-0.029408,-0.027931,0.049549,-0.036386,-0.074033,...,0.013875,0.009445,0.006688,0.011338,-0.006873,0.996964,0.000000,0.0,0.0,0
1,0.158270,0.059584,0.062339,-0.003664,0.098404,-0.104466,-0.070676,-0.037005,-0.090281,0.020334,...,0.016110,0.018466,-0.017619,0.013812,0.025611,0.998269,0.000000,0.0,0.0,0
2,0.199220,0.098726,0.072739,-0.074504,0.055474,-0.163461,-0.010336,-0.067058,0.003318,-0.002771,...,0.012931,-0.002693,-0.040107,-0.023044,-0.035252,0.994044,0.000000,0.0,0.0,0
3,0.170075,0.088968,0.046577,-0.135070,-0.031383,0.104235,-0.005905,-0.029768,0.082635,-0.031929,...,0.032518,-0.004943,-0.001727,0.009609,-0.018234,0.968134,0.030242,0.0,0.0,0
4,0.078412,-0.013877,0.014495,0.029780,0.065738,-0.003143,-0.023877,0.040031,0.027363,-0.021249,...,-0.023550,0.007782,0.033986,-0.015414,-0.022187,0.996865,0.000000,0.0,0.0,0


In [114]:
X = df1.values

In [106]:
#Again, applying machine learning models
X_train, X_test, y_train, y_test = train_test_split(X,merged_df["Journal"],test_size=0.2)

In [ ]:
lr_no_regularization = LogisticRegression(C = 100000000000000)
lr_no_regularization.fit(X_train,y_train)
y_pred= lr_no_regularization.predict(X_test)
logic_accuracy = metrics.accuracy_score(y_test,y_pred)

In [118]:
#RandomForestClassifier
rfmodel = RandomForestClassifier(n_estimators = 500, min_samples_leaf=3, max_features='auto', n_jobs=-1)
rfmodel.fit(X_train,y_train)
rf1_test_pred = rfmodel.predict(X_test)